In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from src.utilities import Matrix

from src.observables import entropy_vn, covariance_invariants, symplectic_eigenvalues, symplectic_eigenvalues_transposed, symplectic_entropy, mutual_information, gaussian_quantum_discord, logarithmic_negativity

SUFFIX = '.npy'

In [ ]:
# Files utilities

def file_dims(filename):
    return int(filename.split('_')[-3][1:])


def file_time(filename):
    return int(filename.split('_')[-2][1:])


def file_timedelta(filename):
    return float(filename.split('_')[-1][2:])


def check_file_metadata(filename, d, dt):
    dims = file_dims(filename)
    timedelta = file_timedelta(filename)
    return dims == d and timedelta == dt


def get_all_files(state, observable):
    files = [file for file in os.listdir(f'../objects/{state}') if file.endswith(SUFFIX) and file.startswith(f'rho_{observable}')]
    return files

def get_last_file(startswith, dims, state):
    all_files = [f.removesuffix(SUFFIX) for f in os.listdir(f'../objects/{state}') if f.startswith(f'{startswith}')]
    timedeltas = [file_timedelta(f) for f in all_files]
    max_times = {dt: max([file_time(f) for f in all_files if check_file_metadata(f, dims, dt)]) for dt in timedeltas}
    files = {dt: [f for f in all_files if check_file_metadata(f, dims, dt) and file_time(f) == max_times[dt]][0] for dt in timedeltas}
    return dict(sorted(files.items()))

def get_cov_of_dims(dims, state) -> dict:
    files = get_last_file(f'rho_covariance_D{dims}', dims, state)
    return files

def get_heat_of_dims(dims, state) -> dict:
    files = get_last_file(f'rho_heats_D{dims}', dims, state)
    return files

In [ ]:
def get_observable(observable, covs, iterator):
    if observable == 'Entropy':
        func = symplectic_entropy
    elif observable == 'Mutual Information':
        func = mutual_information
    elif observable == 'Quantum Discord':
        func = gaussian_quantum_discord
    elif observable == 'Logarithmic Negativity':
        func = logarithmic_negativity
    else:
        raise ValueError(f'Observable {observable} not supported')

    arr = []
    for i in iterator:
        arr.append(func(covs[i]))
    return np.array(arr).real

# Effects of Interaction Time on the evolution

In [ ]:
observables = ['Entropy', 'Mutual Information', 'Quantum Discord', 'Logarithmic Negativity', 'Heat Flux']
df = pd.DataFrame(columns=observables)
state = 'thermal'
dims = 17
cov_files = get_cov_of_dims(dims, state)
heat_transfers = get_heat_of_dims(dims, state)
for timedelta, file in cov_files.items():
    dt_df = pd.DataFrame(columns=observables)
    cov_evolution = np.load(f'../objects/{state}/' + file + SUFFIX)
    heat_transfer = np.load(f'../objects/{state}/' + heat_transfers[timedelta] + SUFFIX).real

    iterator = np.linspace(0, len(cov_evolution) - 1, 1000).astype(int)
    dt_df['Time'] = [i*timedelta for i in iterator]
    dt_df['Interaction Time'] = timedelta

    for obs in observables:
        if obs == 'Heat Flux':
            dt_df['J1'] = [heat_transfer[i, 0] for i in iterator]
            dt_df['J2'] = [heat_transfer[i, 1] for i in iterator]
            dt_df['Jc'] = [heat_transfer[i, 2] for i in iterator]
        else:
            dt_df[obs] = get_observable(obs, cov_evolution, iterator)

    df = pd.concat([df, dt_df])
df.drop('Heat Flux', axis=1, inplace=True)

In [ ]:
fig_entropy = px.line(df, x='Time', y='Entropy', color='Interaction Time', title='Entropy')
fig_entropy.show()

In [ ]:
fig_discord = px.line(df, x='Time', y='Quantum Discord', color='Interaction Time', title='Quantum Discord')
fig_discord.show()

In [ ]:
fig_mutinf = px.line(df, x='Time', y='Mutual Information', color='Interaction Time', title='Mutual Information')
fig_mutinf.show()

In [ ]:
fig_logneg = px.line(df, x='Time', y='Logarithmic Negativity', color='Interaction Time', title='Logarithmic Negativity')
fig_logneg.show()

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Interaction Time'], value_vars=['J1', 'J2', 'Jc'], var_name='Current', value_name='Heat')
fig_heat2 = px.line(df_melt, x='Time', y='Heat', color='Interaction Time', line_dash='Current', title='Heat Currents')
fig_heat2.show()